In [1]:
import sys
sys.path.insert(0, '../')

import numpy as np
from src.time_tagger import TimeTaggerController, TT_Simulator, two_particle_states
from src.kinetic_mount_controller import KineticMountControl
from src.osc import OSCCommunicator, OSCTarget

# OSC Server Setup

In [2]:
my_ip = '128.131.195.250'
my_port = 7401 

if 'OSC' not in globals():
    OSC = OSCCommunicator(my_ip=my_ip, my_port=my_port)

Myself = OSCTarget(ip=my_ip, port=my_port, name='Myself')
Clemens = OSCTarget(ip='192.168.0.3', port=7400, name='Clemens')

# Start the server
OSC.start_server()

OSC Server Started. Serving on ('128.131.195.250', 7401)
--------------------------------------------------------


In [3]:
# Send measurements
measurement_results = [1, 2, 3, 4]
OSC.send_measurement(target=Myself, measurement_results=measurement_results)

Measurement [1, 2, 3, 4] sent to Myself


Received measurement: (1, 2, 3, 4)


# Setup Simulator

In [2]:
SimulatorPhiPlus = TT_Simulator(two_particle_states['phi_plus'], initial_state_noise=0, debug=False)
CHSH_angles = SimulatorPhiPlus.CHSH_angles_for_filters

## Make 1 Measurement and Send to Clemens

In [131]:
#TODO Package random choice stuff into extra function
theta_a = np.random.choice(CHSH_angles[0:2])
theta_b = np.random.choice(CHSH_angles[2:4])

integratedMeasurement = SimulatorPhiPlus.measure_n_entangled_pairs_filter_angles(1000, theta_a, theta_b)
print(f'theta_a={theta_a:.2f}, theta_b={theta_b:.2f}')
print(f'counts: {integratedMeasurement}')

#TODO convert to format clemens wants
effectiveSingleMeasurement = np.random.choice(a=[0, 1, 2, 3], p=integratedMeasurement / integratedMeasurement.sum())

mTranslate = {0 : [1, 1],
              1 : [1, -1],
              2 : [-1, 1],
              3 : [-1, -1]}

resultClemensFormat = [1 if theta_a == CHSH_angles[0] else 2, 1 if theta_b == CHSH_angles[2] else 2 ,*mTranslate[effectiveSingleMeasurement]]
print(f'formatted: {resultClemensFormat}')

#OSC.send_measurement(Clemens, resultClemensFormat)


theta_a=0.00, theta_b=-11.25
counts: [413  80  73 434]
formatted: [1, 1, 1, 1]


Two options for how I could manage getting single measurement results from the real experiment:

1. Integrate counts over a period x, and then choose a random result based on the distribution accross the 4 coincidence options (Gives the expected behaviour but introduces a non-true random choice, but we could sample this randomness from somewhere else. Also have non true random choice of basis anyway). This would be by far the simpler option

2. Could read out RAW time tag data for all 4 coincidence virtual channels and and pick the channel that got the earliest time tag. Would be the more elegant option, but is more prone to systematic errors like not all channels starting at the same time (could mitigate by choosing a threashold wait time)